In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext("local[*]")
spark = SparkSession.builder.getOrCreate()
import pandas as pd  

23/02/25 13:09:37 WARN Utils: Your hostname, ubuntu-hadoop resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/02/25 13:09:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/25 13:09:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/25 13:09:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/25 13:09:40 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
from __future__ import print_function
from pyspark.sql import SparkSession
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml import Pipeline
#evaluation
from pyspark.mllib.evaluation import MultilabelMetrics
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [3]:
def m_metrics_l(ml_model,test_data):
    predictions = ml_model.transform(test_data).cache()
    predictionAndLabels = predictions.select("label","prediction").rdd.map(lambda x: (float(x[0]), float(x[1]))).cache()
    
    # Print some predictions vs labels
    # print(predictionAndLabels.take(10))
    metrics = MulticlassMetrics(predictionAndLabels)
    
    # Overall statistics
    precision = metrics.precision(1.0)
    recall = metrics.recall(1.0)
    f1Score = metrics.fMeasure(1.0)
    print(f"Precision = {precision:.4f} Recall = {recall:.4f} F1 Score = {f1Score:.4f}")
    print("Confusion matrix \n", metrics.confusionMatrix().toArray().astype(int))


In [4]:
# read the PySpark DataFrame from the Parquet file
df_training_m1 = spark.read.parquet("data/transformed_training.parquet")
df_testing_m1 = spark.read.parquet("data/transformed_test.parquet")
df_val_m1 = spark.read.parquet("data/transformed_val.parquet")
df_training_m1.cache()
df_testing_m1.cache()
df_val_m1.cache()
print(df_testing_m1.show(5))

+--------------------+-------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|emotion|label|              text_c|               words|            filtered|         rawFeatures|         featuresIDF|
+--------------------+-------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|im feeling rather...|sadness|    1|  feel rather rot...|[, , feel, rather...|[, , feel, rather...|(1000,[0,1,42,102...|(1000,[0,1,42,102...|
|im updating my bl...|sadness|    1|  update   blog b...|[, , update, , , ...|[, , update, , , ...|(1000,[0,1,117,34...|(1000,[0,1,117,34...|
|i never make her ...|sadness|    1|  never make she ...|[, , never, make,...|[, , never, make,...|(1000,[0,1,2,4,9,...|(1000,[0,1,2,4,9,...|
|i left with my bo...|    joy|    0|  leave with   bo...|[, , leave, with,...|[, , leave, , , b...|(1000,[0,1,39,249...|(1000,[0,1,39,249...|
|i was

In [5]:
# import time
classifier = LinearSVC(maxIter=10, regParam=0.1, featuresCol = "featuresIDF", weightCol="weight", labelCol="label")
# Define OneVsRest strategy
ovr = OneVsRest(classifier=classifier, labelCol="label", featuresCol="featuresIDF", weightCol="weight")
pipeline = Pipeline(stages=[ovr])
model = pipeline.fit(df_training_m1)
m_metrics_l(model,df_val_m1)

23/02/25 13:11:05 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/02/25 13:11:05 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/02/25 13:11:05 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/02/25 13:11:05 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


/home/hicran/Documents/emotions-detector/virtualenv_emotions/lib/python3.10/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Precision = 0.8836 Recall = 0.8556 F1 Score = 0.8694
Confusion matrix 
 [[671  47  43  30  79  23]
 [ 22 486  25  19   6  10]
 [  6   8 205   6   3   2]
 [  3   7   2 157   1  13]
 [  2   2   0   0  88   0]
 [  0   0   0   0   1  33]]


In [8]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", metricName="f1")

# Define the hyperparameter grid
param_grid = ParamGridBuilder() \
    .addGrid(classifier.maxIter, [10,50, 100]) \
    .addGrid(classifier.regParam,  [0.01, 0.1, 1.0])\
    .build()
    
# Define the cross-validator
cv = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)




In [7]:
# Fit the cross-validator to the training set
cv_model = cv.fit(df_training_m1)

# get the best model
best_model = cv_model.bestModel

KeyboardInterrupt: 

In [ ]:
predictions = best_model.transform(df_testing_m1)
f1 = evaluator.evaluate(predictions)
print("f1 on validation set for best model:", f1)